# Imports and setting up viz

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_routing import *

# Load files
File to evaluate the behaviour of native routing compared to standard.

First working on the same topographic grid (0.5°).

4 files are considered : one from the standard routing simulation, three from the native routing simulation (sechiba_history, diag_routing, diag_routing_r)

In [ ]:
dir='../../../JZ_simu_outputs/routing_native'

In [ ]:
#open netcdf files for standard routing
filename = '{}/eval_halfdeg/nat7.1/MO/nat7.1_*_1M_sechiba_history.nc'.format(dir)
std = xr.open_mfdataset(filename)
std

In [ ]:
#open netcdf files for native routing on ORC grid
# filename = 'nat8.1/MO/nat8.1_*_1M_sechiba_history.nc'
filename = '{}/eval_halfdeg/nat14/MO/nat14_*_1M_sechiba_history.nc'.format(dir)
natORC = xr.open_mfdataset(filename)
natORC

In [ ]:
#open netcdf files for native routing diag_routing.nc (on ORC grid)
# filename = 'nat8.1/routing_ORC_grid/nat8.1_*_routing.nc'
filename = '{}/eval_halfdeg/nat14/DA/nat14_*_routing.nc'.format(dir)
natDIAG = xr.open_mfdataset(filename)
natDIAG

In [ ]:
#open netcdf files for native routing diag_routing_r (regridded to 38x36)
# filename = 'nat8.1/routing_native_grid_regridded/nat8.1_*.nc'
filename = '{}/eval_halfdeg/nat14/DA/nat14_*routing_r.nc'.format(dir)
natDIAG_R = xr.open_mfdataset(filename)
natDIAG_R

# File manipulations

In [ ]:
#rename time_counter to time all files
std = std.rename({'time_counter' : 'time'})
natORC = natORC.rename({'time_counter' : 'time'})
natDIAG = natDIAG.rename({'time_counter' : 'time'})
natDIAG_R = natDIAG_R.rename({'time_counter' : 'time'})

In [ ]:
# Remove first years to consider a spinup
spinupyears=[2000,2001,2002]
# spinupyears=[]
for year in spinupyears:
    std = std.where(std['time.year'] != year, drop=True)
    natORC = natORC.where(natORC['time.year'] != year, drop=True)
    natDIAG = natDIAG.where(natDIAG['time.year'] != year, drop=True)
    natDIAG_R = natDIAG_R.where(natDIAG_R['time.year'] != year, drop=True)

In [ ]:
#Rename lat_domain_landpoints to lat and lon_domain_landpoints to lon in natDIAG
# natDIAG = natDIAG.rename({'lat_domain_landpoints':'lat','lon_domain_landpoints':'lon'})

#OR do it for lat_domain_landpoints_routing and lon_domain_landpoints_routing (for reservoirs)

natDIAG = natDIAG.rename({'lat_domain_landpoints_routing':'lat','lon_domain_landpoints_routing':'lon'})
natDIAG

In [ ]:
#remove first and last longitude and latitude for natDIAG_R
natDIAG_R = natDIAG_R.isel(lon=slice(2,-2),lat=slice(2,-2))

In [ ]:
#extract monthly mean from natDIAG_R
natDIAG_month= natDIAG.resample(time='MS').mean()
#shift all values by 14 days to be in the middle of the month
natDIAG_month['time'] = natDIAG_month['time'] + np.timedelta64(14, 'D')
natDIAG_month

In [ ]:
#extract monthly mean from natDIAG_R
natDIAG_R_month= natDIAG_R.resample(time='MS').mean()
#shift all values by 14 days to be in the middle of the month
natDIAG_R_month['time'] = natDIAG_R_month['time'] + np.timedelta64(14, 'D')
natDIAG_R_month

In [ ]:
#Change dataset names attribute for automatic labeling
std.attrs['name'] = 'Standard routing (subgrid_halfdeg)'
natORC.attrs['name'] = 'Routing interp_topo_halfdeg'
natDIAG_R.attrs['name'] = 'Routing interp_topo_halfdeg'
natDIAG_month.attrs['name'] = 'diag_routing_monthly'
natDIAG_R_month.attrs['name'] = 'diag_routing_monthly_r'

In [ ]:
#Rename variables in natDIAG_R_month and natDIAG_R
dict = {'routing_stream_reservoir_r' : 'streamr',
        'routing_fast_reservoir_r' : 'fastr',
        'routing_slow_reservoir_r' : 'slowr',
        'routing_drainage_r' : 'drainage',
        'routing_runoff_r' : 'runoff',
        'routing_hydrographs_r' : 'hydrographs',
        'routing_riverflow_r' : 'riverflow',
        'routing_coastalflow_r' : 'coastalflow'}
#change the variables that exist according to the dictionary
natDIAG_R = natDIAG_R.rename(dict)
natDIAG_R_month = natDIAG_R_month.rename(dict)

In [ ]:
#Rename variables in natDIAG_month
dict = {'routing_stream_reservoir' : 'streamr',
        'routing_fast_reservoir' : 'fastr',
        'routing_slow_reservoir' : 'slowr',
        'routing_drainage' : 'drainage',
        'routing_runoff' : 'runoff',
        'routing_riverflow' : 'riverflow',
        'routing_coastalflow' : 'coastalflow'}
#change the variables that exist according to the dictionary
natDIAG_month = natDIAG_month.rename(dict)
natDIAG = natDIAG.rename(dict)
natDIAG

# Stations

In [ ]:
filename='../../../routing/GRDC_Monthly_Jan20_v1_ES.nc'
stations = xr.open_dataset(filename)
stations

In [ ]:
stations_merit = {
    6298249: {'name': 'Zaragoza',  'river': 'Ebro', 'lon_grid':-0.8749926686286926, 'lat_grid':41.65833282470703, 'year_min':2003, 'year_max':2012},
    6298992: {'name': 'Albarreal De Tajo',  'river': 'Tajo', 'lon_grid':-4.17499303817749, 'lat_grid':39.891666412353516, 'year_min':2003, 'year_max':2012 },
    6298481: {'name': 'San Miguel del Pino', 'river': 'Duero', 'lon_grid':-4.92499303817749, 'lat_grid':41.508331298828125, 'year_min':2003, 'year_max':2012},
    6298259:{'name': 'Badajoz', 'river': 'Guadiana', 'lat_grid': 38.85833358764648, 'lon_grid': -7.008326530456543, 'last_record': '2013-09-15', 'year_min':2003, 'year_max':2012}
                }

In [ ]:
# plot station location on a map with rivers and coastlines
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cartopy.feature.RIVERS)
ax.set_extent([-10, 4, 35, 45])
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
gl.ylocator = gl.ylocator = plt.MaxNLocator(5)
gl.right_labels = False
gl.top_labels = False
for key, coordinates in stations_merit.items():
    plt.scatter(coordinates['lon_grid'], coordinates['lat_grid'], s=30, label=coordinates['name'], marker='o')
plt.title('Location of selected station')
#activate legend and put it in the bottom right corner
plt.legend(loc='lower right')

In [ ]:
# visualize time series for selected stations
fig, ax = plt.subplots(2, 2, figsize=(25, 10))
ax = ax.flatten()
for i, (key, coordinates) in enumerate(stations_merit.items()):
    mask = (stations['number'] == key)
    station = stations.sel(stations=mask)
    #remove years before 2003 and after 2012
    station = station.where((station['time.year'] >= coordinates['year_min']), drop=True)
    station = station.where((station['time.year'] <= coordinates['year_max']), drop=True)
    ax[i].plot(station['time'],station['mergedhydro'], label='obs', c='black')
    ax[i].set_title(coordinates['name'])
    ax[i].set_ylabel('discharge [m3/s]')
    ax[i].legend()

# Maps

In [ ]:
#Display map of var for ds
var='fastr'
ds = natDIAG_R_month
valmax=1
plotvar = ds[var].mean(dim='time')
plotvar=plotvar.where(plotvar>1e-5)
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
#use colormap with 20 bins from min to max value
plotvar.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=myvir, vmax=valmax)
plt.title(var + ' (' + ds.name + ')')

In [ ]:
# Display map of var
var = 'hydrographs'
max_val=600

ds_std = std
plotvar_std = ds_std[var].mean(dim='time')
plotvar_std=plotvar_std.where(plotvar_std>0)

ds_nat = natDIAG_R
plotvar_nat = ds_nat[var].mean(dim='time')
plotvar_nat=plotvar_nat.where(plotvar_nat>0)

# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(16, 6), subplot_kw={'projection': ccrs.PlateCarree()})
# fig.suptitle(var + ' average over the period (2003-2012)')
fig.suptitle(ds_std[var].long_name + ' average over the period (2003-2012, {})'.format(ds_std[var].units))

# Plot for std dataset
axs[0].coastlines()
plotvar_std.plot(ax=axs[0], transform=ccrs.PlateCarree(), cmap=reds, vmax=max_val)
axs[0].set_title(f'{ds_std.name}')

# Plot for natORC dataset
axs[1].coastlines()
plotvar_nat.plot(ax=axs[1], transform=ccrs.PlateCarree(), cmap=reds, vmax=max_val)
axs[1].set_title(f'{ds_nat.name}')

In [ ]:
#RMSE map
var='fastr'
ds1=std
ds2=natORC
#compute RMSE between ds1 and ds2
rmse = np.sqrt(((ds1[var]-ds2[var])**2).mean(dim='time'))
rel_rmse = rmse/ds2[var].mean(dim='time')

#output the value of the rmse after spatial averaging over all grid points
print(var + ' RMSE (' + ds1.name + '-' + ds2.name+ ', spatial and temporal mean) : ' + str(rmse.mean(dim=['lon','lat']).values))

#display map of rmse
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
rmse.where(rmse>0).plot(ax=ax, transform=ccrs.PlateCarree(), cmap=reds)
# plt.title(ds1[var].long_name + ' average RMSE  (' + ds1.name + ' - ' + ds2.name + ')')
plt.title(ds1[var].long_name + ' average RMSD')

In [ ]:
#Diff map
var='hydrographs'
ds1=natDIAG_R
ds2=std
maxval=None
minval=None
#compute diff between ds1 and ds2
diff = (ds1[var]-ds2[var]).mean(dim='time')
# rel_diff = ((ds1[var]-ds2[var]) / (ds2[var] + 1E-16)).mean(dim='time')
# Transform rel_diff using log10
# rel_diff_log = np.log10(rel_diff)
#display map
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
# rel_diff.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=emb)
diff.where(diff!=0).plot(ax=ax, transform=ccrs.PlateCarree(), cmap=emb, vmin=minval, vmax=maxval)
plt.title(ds2[var].long_name + ' difference')# (' + ds1.name + ' - ' + ds2.name + ')')

In [ ]:
#RMSE and rRMSE values for multiple variables
varlist=['streamr', 'fastr', 'slowr']
# varlist=['drainage', 'runoff']
ds1=natORC
ds2=std
#compute RMSE between ds1 and ds2
for var in varlist:
    rmse = np.sqrt(((ds1[var]-ds2[var])**2).mean(dim='time'))
    # rel_rmse = rmse/ds2[var].mean(dim='time')
    #output the value of the rmse after spatial averaging over all grid points
    print(var + ' RMSE (' + ds1.name + '-' + ds2.name+ ', spatial and temporal mean) : ' + str((rmse.mean(dim=['lon','lat']).values)))
    # print(var + ' rRMSE (' + ds1.name + '-' + ds2.name+ ', spatial and temporal mean) : ' + str((rel_rmse.mean(dim=['lon','lat']).values)))

# Time series

In [ ]:
#Average time series
var='slowr'
year_min=2000
year_max=2012
ds1=std
ds1 = ds1.where(ds1['time.year'] >= year_min, drop=True).where(ds1['time.year'] <= year_max, drop=True)
#plot the time series of hydrographs at the point (lon,lat)
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()
ax.grid()
ds1[var].mean(dim=['lon','lat']).plot(ax=ax)
plt.title(var + ' (' + ds1.name + ')')
plt.legend()

# Hydrographs en un point

In [ ]:
river_coordinates = {
    'Ebre': {'lon': 0.75, 'lat': 40.75},
    'Tage': {'lon': -9.25, 'lat': 39.25},
    'Douro': {'lon': -8.75, 'lat': 41.25},
    'Guadiana': {'lon': -7.25, 'lat': 38.25},
    'Guadalquivir': {'lon': -6.25, 'lat': 36.75},
    'Minho': {'lon': -8.75, 'lat': 42.25}
}

In [ ]:
#Time series
year_min=2003
year_max=2011
# Douro
lon=-8.75
lat=41.25

var="hydrographs"
ds1=std
ds1 = ds1.where(ds1['time.year'] >= year_min, drop=True).where(ds1['time.year'] <= year_max, drop=True)
ds2=natDIAG_R
ds2 = ds2.where(ds2['time.year'] >= year_min, drop=True).where(ds2['time.year'] <= year_max, drop=True)
#plot the time series of hydrographs at the point (lon,lat)
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()
ax.grid()
ax.set_ylabel('hydrographs')
ds1[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds1.name)
ds2[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds2.name)
# ds2['runoff'].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label='runoff')
plt.title('hydrographs at (' + str(lon) + ',' + str(lat) + '), m³/s')
plt.legend()

In [ ]:
#Time series
year_min=2010
year_max=2011
ds1=std
ds1 = ds1.where(ds1['time.year'] >= year_min, drop=True).where(ds1['time.year'] <= year_max, drop=True)
#plot the time series of hydrographs at the point (lon,lat)
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()
ax.grid()
ds1['runoff'].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label='runoff',c='green')
ds1['drainage'].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label='drainage',c='brown')
ds1['rain'].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label='rainfall',c='blue')
plt.title('Runoff and drainage at (' + str(lon) + ',' + str(lat) + '), mm/d')
plt.legend()

In [ ]:
#Cycle saisonnier (moyenne par jour de l'année)
var="hydrographs"
#Ebre
# lon=0.75
# lat=40.75
# # Tage
# lon=-9.25
# lat=39.25
# # Douro
# lon=-8.75
# lat=41.25
# # Guadiana
lon=-7.25
lat=38.25
# # Guadalquivir
# lon=-6.25
# lat=36.75
ds1=std
ds2=natDIAG_R
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()
ax.grid()
ax.set_xlabel('month')
ax.set_ylabel('hydrographs')
ax.set_title('hydrographs at (' + str(lon) + ',' + str(lat) + ')')
#compute the annual cycle of hydrographs at the point (lon,lat)
ds1[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.dayofyear').mean(dim='time').plot(ax=ax, label=ds1.name)
ds2[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.dayofyear').mean(dim='time').plot(ax=ax, label=ds2.name)
plt.legend()

In [ ]:
#cycle saisonnier pour 6 points de grille
var = "hydrographs"
ds1 = std
ds2 = natDIAG_R

# Create subplots for each river in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over rivers and plot hydrographs
for i, (river, coordinates) in enumerate(river_coordinates.items()):
    lon = coordinates['lon']
    lat = coordinates['lat']
    
    ax = axes[i]

    # Compute the annual cycle of hydrographs at the point (lon, lat) for each dataset
    ds1[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds1.name)
    ds2[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds2.name)
    
    ax.legend()
    #set subplot title to river name
    ax.set_title(river)

# Remove any remaining empty subplots
for i in range(len(river_coordinates), len(axes)):
    fig.delaxes(axes[i])

plt.suptitle("River discharge seasonnal cycle (m³/s, 2003-2012)")
plt.tight_layout()

In [ ]:
#cycle saisonnier pour 6 stations
var = "hydrographs"
ds1 = std
ds2 = natDIAG_R_month

# Create subplots for each river in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over rivers and plot hydrographs
# for i, (station, coordinates) in enumerate(stations_merit.items()):
#     lon = coordinates['lon']
#     lat = coordinates['lat']
    
#     ax = axes[i]

#     # Compute the annual cycle of hydrographs at the point (lon, lat) for each dataset
#     ds1[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds1.name)
#     ds2[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds2.name)
    
#     ax.legend()
#     #set subplot title to river name
#     ax.set_title(river)

for i, (key, coordinates) in enumerate(stations_merit.items()):
    ax=axes[i]
    mask = (stations['number'] == key)
    station = stations.sel(stations=mask)
    #remove years before 2003 and after 2012
    station = station.where((station['time.year'] >= coordinates['year_min']), drop=True)
    station = station.where((station['time.year'] <= coordinates['year_max']), drop=True)
    ax.plot(station['time'],station['mergedhydro'], label='obs', c='black')

    lon = coordinates['lon_grid']
    lat = coordinates['lat_grid']
    ds1[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds1.name)
    ds2[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds2.name)

    ax.set_title(coordinates['name'])
    ax.set_ylabel('discharge [m3/s]')
    ax.legend()

# Remove any remaining empty subplots
for i in range(len(river_coordinates), len(axes)):
    fig.delaxes(axes[i])

plt.suptitle("River discharge time series (m³/s)")
plt.tight_layout()

In [ ]:
#cycle saisonnier pour 6 stations
var = "hydrographs"
ds1 = std
ds2 = natDIAG_R

# Create subplots for each river in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over rivers and plot hydrographs
# for i, (station, coordinates) in enumerate(stations_merit.items()):
#     lon = coordinates['lon']
#     lat = coordinates['lat']
    
#     ax = axes[i]

#     # Compute the annual cycle of hydrographs at the point (lon, lat) for each dataset
#     ds1[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds1.name)
#     ds2[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds2.name)
    
#     ax.legend()
#     #set subplot title to river name
#     ax.set_title(river)

for i, (key, coordinates) in enumerate(stations_merit.items()):
    ax=axes[i]

    lon = coordinates['lon_grid']
    lat = coordinates['lat_grid']
    ds1[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds1.name)
    ds2[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds2.name)


    mask = (stations['number'] == key)
    station = stations.sel(stations=mask)
    #remove years before 2003 and after 2012
    station = station.where((station['time.year'] >= coordinates['year_min']), drop=True)
    station = station.where((station['time.year'] <= coordinates['year_max']), drop=True)
    station = station.groupby('time.month').mean()
    ax.plot(station['month'],station['mergedhydro'], label='obs', c='black')

    ax.set_title(coordinates['name'])
    ax.set_ylabel('discharge [m3/s]')
    #add months name to x-axis ticks
    months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax.set_xticks(np.arange(1,13))
    ax.set_xticklabels(months)
    ax.legend()

# Remove any remaining empty subplots
for i in range(len(river_coordinates), len(axes)):
    fig.delaxes(axes[i])

plt.suptitle("River discharge seasonnal cycle (m³/s, 2003-2012)")
plt.tight_layout()